In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import sqlite3

%matplotlib inline
#bommovies dataframe this is bad data
bom_movies_df = pd.read_csv('../data/bom.movie_gross.csv.gz')
#imbd dataframe
con = sqlite3.connect('../data/im.db') 
cursor = con.cursor()
imdb_schema_df = pd.read_sql('''
SELECT 
    *
FROM
    sqlite_master
''', con)   
#imdb directors df
directors_df = pd.read_sql('''select * from directors''',con)
#imdb writers df
writers_df = pd.read_sql('''select * from writers''',con)
#imdb know for df
known_for_df = pd.read_sql('''select * from known_for''',con)
#imdb principals df
principals_df = pd.read_sql('''select * from principals''',con)
#imdb persons df
persons_df = pd.read_sql('''select * from persons''',con)
#imdb movie basics df
movie_basics_df = pd.read_sql('''select * from movie_basics''',con)
#imdb movie ratings df
movie_ratings_df = pd.read_sql('''select * from movie_ratings''',con)
#imdb movie akas df
movie_akas_df = pd.read_sql('''select * from movie_akas''',con)
#rotten tomatoes movies df this is bad data
rt_movies_df = pd.read_csv('../data/rt.movie_info.tsv.gz', 
                               sep='\t') # sep='\t' needs to be specified since is a .tsv file
#rotten tomatoes reviews df this is bad data
rt_reviews_df = pd.read_csv('../data/rt.reviews.tsv.gz', 
                                sep='\t', # sep='\t' needs to be specified since is a .tsv file
                                encoding='unicode_escape') # using default utf-8 encoding ran into issues with this file
# TheMovieDB movie information table
tmdb_movies_df = pd.read_csv('../data/tmdb.movies.csv.gz')
# The Numbers movie information table
tn_movies_df = pd.read_csv('../data/tn.movie_budgets.csv.gz')

In [2]:
tn_movies_df.info() #  create ROI metric

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [3]:
tmdb_movies_df = tmdb_movies_df.set_index('id') #setting indexs and dropping columns that are not relevant 

In [4]:
movie_basics_df = movie_basics_df.set_index('movie_id')

In [5]:
tmdb_movies_df = tmdb_movies_df.drop('original_language', axis=1)

In [6]:
tmdb_movies_df = tmdb_movies_df.drop('title', axis=1)

In [7]:
tmdb_movies_df = tmdb_movies_df.drop('Unnamed: 0', axis=1)

In [8]:
tmdb_movies_df = tmdb_movies_df.drop('vote_count', axis=1)

In [9]:
tmdb_movies_df = tmdb_movies_df.drop('popularity', axis=1)

In [10]:
tmdb_movies_df = tmdb_movies_df.drop('vote_average', axis=1)

In [11]:
movie_basics_df = movie_basics_df.drop('runtime_minutes', axis=1)

In [12]:
movie_basics_df = movie_basics_df.drop('original_title', axis=1)

In [13]:
tmdb_movies_df['release_date'] = pd.to_datetime(tmdb_movies_df['release_date'], format = '%Y%') 

In [14]:
tmdb_movies_df['year'] = tmdb_movies_df['release_date'].dt.year

In [15]:
tmdb_movies_df = tmdb_movies_df.drop('release_date', axis=1)

In [16]:
tmdb_movies_df = tmdb_movies_df.set_index('original_title')

In [17]:
movie_basics_df = movie_basics_df.set_index('primary_title')

In [18]:
tn_movies_df = tn_movies_df.set_index('movie')

In [19]:
tn_movies_df = tn_movies_df.drop('id', axis = 1)

In [20]:
genre_df = movie_basics_df.join(tmdb_movies_df, how = 'inner') #joining genre databases

In [21]:
final_df = genre_df.join(tn_movies_df, how='inner') #joining genre databases with our the numbers database aka money

In [22]:
final_df.info()  # I need to switch the date to datetime

<class 'pandas.core.frame.DataFrame'>
Index: 4202 entries, #Horror to xXx: Return of Xander Cage
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   start_year         4202 non-null   int64 
 1   genres             4133 non-null   object
 2   genre_ids          4202 non-null   object
 3   year               4202 non-null   int64 
 4   release_date       4202 non-null   object
 5   production_budget  4202 non-null   object
 6   domestic_gross     4202 non-null   object
 7   worldwide_gross    4202 non-null   object
dtypes: int64(2), object(6)
memory usage: 295.5+ KB


In [23]:
final_df = final_df.drop('year', axis=1) #dropping duplicate dates

In [24]:
final_df = final_df.drop('start_year', axis=1)

In [25]:
final_df = final_df[final_df.worldwide_gross != '$0'] #dropping worldwide gross values that equal 0

In [26]:
final_df.info() #I need to change release_date to datetime, gross and budget columns to int

<class 'pandas.core.frame.DataFrame'>
Index: 3669 entries, 10 Cloverfield Lane to xXx: Return of Xander Cage
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   genres             3605 non-null   object
 1   genre_ids          3669 non-null   object
 2   release_date       3669 non-null   object
 3   production_budget  3669 non-null   object
 4   domestic_gross     3669 non-null   object
 5   worldwide_gross    3669 non-null   object
dtypes: object(6)
memory usage: 200.6+ KB


In [27]:
final_df = final_df.dropna() #dropping empty genre values

In [28]:
final_df = final_df.drop_duplicates() #dropping exact duplicates from df, still need to merge or drop duplicates that are not exact

In [29]:
final_df.info() # I need to create columns for each genre and verify over boolean where to add the genre values that are not relevant, release_date needs to be type.datetime and production, domestic, and worldwide need to be ints

<class 'pandas.core.frame.DataFrame'>
Index: 2974 entries, 10 Cloverfield Lane to xXx: Return of Xander Cage
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   genres             2974 non-null   object
 1   genre_ids          2974 non-null   object
 2   release_date       2974 non-null   object
 3   production_budget  2974 non-null   object
 4   domestic_gross     2974 non-null   object
 5   worldwide_gross    2974 non-null   object
dtypes: object(6)
memory usage: 162.6+ KB


In [30]:
final_df['genre_ids'] = final_df["genre_ids"].str.split("," , n = 1, expand = False)

In [31]:
final_df.head()

,genres,genre_ids,release_date,production_budget,domestic_gross,worldwide_gross
10 Cloverfield Lane,"Drama,Horror,Mystery","[[53, 878, 18]]","Mar 11, 2016","$5,000,000","$72,082,999","$108,286,422"
10 Days in a Madhouse,Drama,[[18]],"Nov 11, 2015","$12,000,000","$14,616","$14,616"
12 Strong,"Action,Drama,History","[[10752, 18, 36, 28]]","Jan 19, 2018","$35,000,000","$45,819,713","$71,118,378"
12 Years a Slave,"Biography,Drama,History","[[18, 36]]","Oct 18, 2013","$20,000,000","$56,671,993","$181,025,343"
127 Hours,"Adventure,Biography,Drama","[[12, 18, 53]]","Nov 5, 2010","$18,000,000","$18,335,230","$60,217,171"


In [32]:
final_df = final_df.drop('genres', axis=1) # dropping genres column and using genre_ids as that one seems to place the most relevant genre first

In [33]:
final_df.head()

,genre_ids,release_date,production_budget,domestic_gross,worldwide_gross
10 Cloverfield Lane,"[[53, 878, 18]]","Mar 11, 2016","$5,000,000","$72,082,999","$108,286,422"
10 Days in a Madhouse,[[18]],"Nov 11, 2015","$12,000,000","$14,616","$14,616"
12 Strong,"[[10752, 18, 36, 28]]","Jan 19, 2018","$35,000,000","$45,819,713","$71,118,378"
12 Years a Slave,"[[18, 36]]","Oct 18, 2013","$20,000,000","$56,671,993","$181,025,343"
127 Hours,"[[12, 18, 53]]","Nov 5, 2010","$18,000,000","$18,335,230","$60,217,171"


In [34]:
type(final_df['genre_ids'])

pandas.core.series.Series

In [35]:
final_df['Genre'] = final_df.genre_ids.apply(lambda x: x[0])

In [36]:
final_df.head()

,genre_ids,release_date,production_budget,domestic_gross,worldwide_gross,Genre
10 Cloverfield Lane,"[[53, 878, 18]]","Mar 11, 2016","$5,000,000","$72,082,999","$108,286,422",[53
10 Days in a Madhouse,[[18]],"Nov 11, 2015","$12,000,000","$14,616","$14,616",[18]
12 Strong,"[[10752, 18, 36, 28]]","Jan 19, 2018","$35,000,000","$45,819,713","$71,118,378",[10752
12 Years a Slave,"[[18, 36]]","Oct 18, 2013","$20,000,000","$56,671,993","$181,025,343",[18
127 Hours,"[[12, 18, 53]]","Nov 5, 2010","$18,000,000","$18,335,230","$60,217,171",[12


In [37]:
final_df.head()

,genre_ids,release_date,production_budget,domestic_gross,worldwide_gross,Genre
10 Cloverfield Lane,"[[53, 878, 18]]","Mar 11, 2016","$5,000,000","$72,082,999","$108,286,422",[53
10 Days in a Madhouse,[[18]],"Nov 11, 2015","$12,000,000","$14,616","$14,616",[18]
12 Strong,"[[10752, 18, 36, 28]]","Jan 19, 2018","$35,000,000","$45,819,713","$71,118,378",[10752
12 Years a Slave,"[[18, 36]]","Oct 18, 2013","$20,000,000","$56,671,993","$181,025,343",[18
127 Hours,"[[12, 18, 53]]","Nov 5, 2010","$18,000,000","$18,335,230","$60,217,171",[12


In [38]:
final_df['Genre'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',
value=r'') #cleaning the new Genres tab

In [39]:
final_df.head()

,genre_ids,release_date,production_budget,domestic_gross,worldwide_gross,Genre
10 Cloverfield Lane,"[[53, 878, 18]]","Mar 11, 2016","$5,000,000","$72,082,999","$108,286,422",53
10 Days in a Madhouse,[[18]],"Nov 11, 2015","$12,000,000","$14,616","$14,616",18
12 Strong,"[[10752, 18, 36, 28]]","Jan 19, 2018","$35,000,000","$45,819,713","$71,118,378",10752
12 Years a Slave,"[[18, 36]]","Oct 18, 2013","$20,000,000","$56,671,993","$181,025,343",18
127 Hours,"[[12, 18, 53]]","Nov 5, 2010","$18,000,000","$18,335,230","$60,217,171",12


In [40]:
final_df = final_df.drop('genre_ids', axis =1)

In [41]:
final_df.head()

,release_date,production_budget,domestic_gross,worldwide_gross,Genre
10 Cloverfield Lane,"Mar 11, 2016","$5,000,000","$72,082,999","$108,286,422",53
10 Days in a Madhouse,"Nov 11, 2015","$12,000,000","$14,616","$14,616",18
12 Strong,"Jan 19, 2018","$35,000,000","$45,819,713","$71,118,378",10752
12 Years a Slave,"Oct 18, 2013","$20,000,000","$56,671,993","$181,025,343",18
127 Hours,"Nov 5, 2010","$18,000,000","$18,335,230","$60,217,171",12


In [44]:
final_df['Action'] = final_df['Genre'].isin(['28'])

In [45]:
final_df['Adventure'] = final_df['Genre'].isin(['12'])

In [46]:
final_df['Animation'] = final_df['Genre'].isin(['16'])

In [47]:
final_df['Comedy'] = final_df['Genre'].isin(['35'])

In [48]:
final_df['Crime'] = final_df['Genre'].isin(['80'])

In [49]:
final_df['Documentary'] = final_df['Genre'].isin(['99'])

In [50]:
final_df['Drama'] = final_df['Genre'].isin(['18'])

In [51]:
final_df['Family'] = final_df['Genre'].isin(['10751'])

In [52]:
final_df['Fantasy'] = final_df['Genre'].isin(['14'])

In [53]:
final_df['History'] = final_df['Genre'].isin(['36'])

In [54]:
final_df['Horror'] = final_df['Genre'].isin(['27'])

In [56]:
final_df['Mystery'] = final_df['Genre'].isin(['9648'])

In [57]:
final_df['Music'] = final_df['Genre'].isin(['10402'])

In [58]:
final_df['Romance'] = final_df['Genre'].isin(['10749'])

In [59]:
final_df['SciFi'] = final_df['Genre'].isin(['878'])

In [60]:
final_df['TV_Movie'] = final_df['Genre'].isin(['10770'])

In [61]:
final_df['Thriller'] = final_df['Genre'].isin(['53'])

In [62]:
final_df['War'] = final_df['Genre'].isin(['10752'])

In [63]:
final_df['Western'] = final_df['Genre'].isin(['37'])

In [65]:
final_df.head()

,release_date,production_budget,domestic_gross,worldwide_gross,Genre,Action,Adventure,Animation,Comedy,Crime,...,History,Horror,Music,Mystery,Romance,SciFi,TV_Movie,Thriller,War,Western
10 Cloverfield Lane,"Mar 11, 2016","$5,000,000","$72,082,999","$108,286,422",53,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
10 Days in a Madhouse,"Nov 11, 2015","$12,000,000","$14,616","$14,616",18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12 Strong,"Jan 19, 2018","$35,000,000","$45,819,713","$71,118,378",10752,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
12 Years a Slave,"Oct 18, 2013","$20,000,000","$56,671,993","$181,025,343",18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
127 Hours,"Nov 5, 2010","$18,000,000","$18,335,230","$60,217,171",12,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [69]:
final_df['Adventure'].value_counts()

False    2825
True      149
Name: Adventure, dtype: int64

In [68]:
final_df.describe()

,release_date,production_budget,domestic_gross,worldwide_gross,Genre,Action,Adventure,Animation,Comedy,Crime,...,History,Horror,Music,Mystery,Romance,SciFi,TV_Movie,Thriller,War,Western
count,2974,2974,2974,2974,2974,2974,2974,2974,2974,2974,...,2974,2974,2974,2974,2974,2974,2974,2974,2974,2974
unique,733,276,1596,1658,20,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,"Mar 27, 2015","$5,000,000",$0,"$385,997,896",18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,68,166,114,65,733,2600,2825,2905,2567,2859,...,2946,2717,2963,2945,2875,2881,2973,2740,2959,2964
